# Welcome to the Sentinel Satellite API Tutorial

#### This tutorial will walk you through how to query and download satellite images directly in Python using the SentinelAPI library. If you have any questions, message me (Jake Snow) on Slack. 

### Important Note - In order to follow this tutorial, you need an account with the Copernicus Open Access Hub found at - https://scihub.copernicus.eu/dhus/#/home . If you have never used the Sentinel Satellite before, I suggest reviewing the tutorial put together by Jalen Morgan at the following link - https://byu.app.box.com/file/740506393850

## Step 1 - Installing and Importing Libraries

#### It's likely you don't have the following libraries installed. To install the libraries on your current runtime, simply un-comment out the following cell and run it. If you are planning on using these libraries frequently, I suggest opening up an Anaconda Prompt, copying each line from the following cell individually minus the exclamation point, and running it in the prompt to install the libraries on your system. 

In [3]:
"""
!pip install sentinelsat
!pip install geopandas
!pip install patool
!pip install rasterio
"""

'\n!pip install sentinelsat\n!pip install geopandas\n!pip install patool\n!pip install rasterio\n'

In [1]:
from sentinelsat import SentinelAPI
import geopandas as gpd

#These two libraries are for unzipping and loading downloaded images
import patoolib
import rasterio

## Step 2 - Connect to the API

#### Here we connect to the API by putting in your username and password. 

In [ ]:
user = YOUR_USERNAME 
password = YOUR_PASSWORD
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

## Step 3 - Make a Query

#### This step can be as easy or as complicated as you want to make it. The following query is pretty basic but should be good enough for anything you will probably need to use the API for. A more advanced tutorial can be found here - https://scihub.copernicus.eu/twiki/do/view/SciHubUserGuide/FullTextSearch?redirectedfrom=SciHubUserGuide.3FullTextSearch            but it should be noted that this tutorial is for queries made on the actual Copernicus Hub website and not the Python API so I'm not 100% sure how much crossover there is between the two. Another option is to run ?api.query in an empty cell and read the function documentation.

#### So let's walk through what's going on in this particular query - 

a) footprint - Footprint is the area you want to look at. By inputing a specific coordinate point, all query results will be image that contain the specified point.

b) date - The date range you want to look at. See note below.

c) platformname - Here you have 3 options - Sentinel-1, Sentinel-2, Sentinel-3. For reasons beyond the scope of this tutorial, you should probably always use Sentinel-2.

d) processinglevel - Again, beyond the scope of this tutorial, probably always use Level-2A

e) cloudcoverpercentage - Pretty self explanatory. This will be the range of the cloud cover percentage you want to appear in the results. Note - if using Sentinel-2, you will have access to infared bands which do a pretty good job of cutting through cloud cover, so it might be best to just look at 0 - 100% cloud cover and see what you get since clouds aren't a huge deal.



## Very Important Note! 

### The image database does not keep all satellite images available online. This means that the majority of images are offline. I don't know exaclty how long images are kept online, but in my experience it seems to be about a month.

#### According to the API documentation, if you try to download an offline image, a request for that image to be made available is automatically submitted. Requests are typically fulfilled within 30 minutes. I HAVE YET TO CONFIRM THIS. It should also be noted that even though the request is submitted, you will still get an error (an HTTP error if I remember correctly).

In [7]:
products = api.query(footprint = "intersects(50.2000, -111.6000)",
                     date = ('20210101', '20210626'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,100)
                    )

The format of the products received from a query is a little weird, so putting them into a geopandas dataframe is a good idea

In [8]:
products_gdf = api.to_geodataframe(products)

#Optional - sort the results in order of ascending cloud cover percentage
#products_gdf = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])

products_gdf.head()

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
8df8f4fa-ce23-473b-800e-9c0c1e3d9ba7,S2B_MSIL2A_20210624T181919_N0300_R127_T12UVA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-24T18:19:19.024Z, Instrument: MS...",false,2021-06-24 21:08:58,2021-06-24 18:19:19.024,2021-06-24 18:19:19.024,2021-06-25 05:36:21.329,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS1_20210624T210858_S2021...,S2B_OPER_MSI_L2A_TL_VGS1_20210624T210858_A0224...,S2B_MSIL2A_20210624T181919_N0300_R127_T12UVA_2...,8df8f4fa-ce23-473b-800e-9c0c1e3d9ba7,"MULTIPOLYGON (((-112.08771 49.55808, -110.8650..."
94358710-7aa2-4353-90d8-a5146b845a10,S2A_MSIL2A_20210622T182921_N0300_R027_T12UVA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-22T18:29:21.024Z, Instrument: MS...",false,2021-06-22 22:22:18,2021-06-22 18:29:21.024,2021-06-22 18:29:21.024,2021-06-23 01:13:28.865,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS4_20210622T222218_S2021...,S2A_OPER_MSI_L2A_TL_VGS4_20210622T222218_A0313...,S2A_MSIL2A_20210622T182921_N0300_R027_T12UVA_2...,94358710-7aa2-4353-90d8-a5146b845a10,"MULTIPOLYGON (((-112.38324 49.55649, -110.8650..."
a08fcd07-c834-4dd0-b7e8-f5528ba9e84d,S2A_MSIL2A_20210619T181921_N0300_R127_T12UVA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-19T18:19:21.024Z, Instrument: MS...",false,2021-06-19 22:52:02,2021-06-19 18:19:21.024,2021-06-19 18:19:21.024,2021-06-20 02:30:01.759,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS2_20210619T225202_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210619T225202_A0312...,S2A_MSIL2A_20210619T181921_N0300_R127_T12UVA_2...,a08fcd07-c834-4dd0-b7e8-f5528ba9e84d,"MULTIPOLYGON (((-112.07750 49.55814, -110.8650..."
de2d1ea9-9f99-47c5-9ff4-7c13b338a177,S2B_MSIL2A_20210617T182919_N0300_R027_T12UVA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-17T18:29:19.024Z, Instrument: MS...",false,2021-06-17 22:32:09,2021-06-17 18:29:19.024,2021-06-17 18:29:19.024,2021-06-18 01:12:43.213,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210617T223209_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210617T223209_A0223...,S2B_MSIL2A_20210617T182919_N0300_R027_T12UVA_2...,de2d1ea9-9f99-47c5-9ff4-7c13b338a177,"MULTIPOLYGON (((-112.38324 49.55649, -110.8650..."
636f12cc-629c-4d4f-9997-8bc0893717ac,S2B_MSIL2A_20210614T181919_N0300_R127_T12UVA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-14T18:19:19.024Z, Instrument: MS...",false,2021-06-14 22:24:02,2021-06-14 18:19:19.024,2021-06-14 18:19:19.024,2021-06-15 01:53:38.220,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20210614T222402_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20210614T222402_A0223...,S2B_MSIL2A_20210614T181919_N0300_R127_T12UVA_2...,636f12cc-629c-4d4f-9997-8bc0893717ac,"MULTIPOLYGON (((-112.09299 49.55805, -110.8650..."


## Step 4 - Download images

#### You can either download a single image or download every image in a query using api.download or api.download_all respectively. 

#### Important Note - The data downloaded is large. Each product is usually around 1 GB in size. For me with my computer, it usually takes around 30 minutes to download one product. 

#### Something to explore - On the API documentation page linked above, there is mention of a way of downloading only certain bands instead of downloading everything. This would greatly improve download time and save on space, but it's really confusing and I haven't been able to figure out how to use it. It would definitely be worth exploring.

In [ ]:
i = list(products_gdf.index)[0]

#It's important to check if the product is available to download first. If it isn't, you could either forget it or
#you could try to download it in order to request that the product be made available. 
#For more information, view the official API documentation or read the note under step 3.

if api.is_online(id):
    api.download(i)
else:
    print("Product is offline. View comments above.")
    
#Like I mentioned above, another option is - 

#api.download_all(products)

### And that's it! In a future tutorial, I will explain how to load images and calculate indices.